EMERGENT uses several unique datatypes to organize data. In this tutorial we will examine the various functionalities provided by these new types.

# DataDict
The DataDict class is similar to the collections library's OrderedDict: it preserves the order in which elements are added. However, it also implements several methods used in the REST API standard. Let's prepare a database:

In [1]:
from emergent.utilities.containers import DataDict

database = DataDict({'alphabet': {'a': 1, 'b': 2, 'z': 26}})

Because Python dicts are mutable and assignment is done by reference, copying a dict is not as simple as assigning a new one:

In [2]:
d = {'a':1, 'b':2}
D = d
D['a'] = 5
print(d)
print(D)

{'a': 5, 'b': 2}
{'a': 5, 'b': 2}


If we want to create an unlinked copy, we can use the APIDict.copy() method:

In [3]:
d = DataDict({'a':1, 'b':2})
D = d.copy()
D['a'] = 5
print(d)
print(D)

DataDict([('a', 1), ('b', 2)])
DataDict([('a', 5), ('b', 2)])


To simplfy update an existing field, we can call the patch() method:

In [4]:
db = database.copy()
print('Copy:', db)
new = {'alphabet': {'a': 2, 'c': 3}, 'numbers': {'1': 1, '2': 2}}
print('New addition:', new)
db.patch(new)
print('Patched:', db)

Copy: DataDict([('alphabet', {'a': 1, 'b': 2, 'z': 26})])
New addition: {'alphabet': {'a': 2, 'c': 3}, 'numbers': {'1': 1, '2': 2}}
Patched: DataDict([('alphabet', {'a': 2, 'b': 2, 'z': 26})])


Since the new addition only contains one field ('c' in the 'alphabet' subdict) which overlaps with the database, this is the only value that is updated. 

The put() method lets us update existing fields _as well as_ adding new fields. Let's try it out on a fresh copy:



In [5]:
db = database.copy()
db.put(new)
print('Put:', db.as_dict())

Put: {'alphabet': {'a': 2, 'b': 2, 'z': 26, 'c': 3}, 'numbers': {'1': 1, '2': 2}}


The reduce() method allows us to get a version of the database containing only the specified endpoint. For instance, suppose we have a database specifying the state and min/max range of all knobs:

In [13]:
database = DataDict({'hub': {'thing1': {'X': {'state': 2.718, 'min': 0, 'max': 5},
                                      'Y': {'state': 3.14, 'min': 3, 'max': 4}
                                      },
                             'thing2': {'u': {'state': 42, 'min': 40, 'max': 50}}
                           }
                   }
                  )

We can use the find() method to extract only the parts referring to state:

In [14]:
state = database.find('state')
print(state)

DataDict([('hub', {'thing1': {'X': 2.718, 'Y': 3.14}, 'thing2': {'u': 42}})])


<div class="alert alert-block alert-danger">
Warning: find() may produce unexpected results if the same dict key exists across different levels of nesting!
</div>

The get() method is left the same as in the usual dict class, allowing objects at various levels to be accessed through iterative get() calls:

In [12]:
print(database.get('hub'))
print(database.get('hub').get('thing1').get('X').get('state'))

{'thing1': {'X': {'state': 2.718, 'min': 0, 'max': 5}, 'Y': {'state': 3.14, 'min': 3, 'max': 4}}, 'thing2': {'u': {'state': 42, 'min': 40, 'max': 50}}}
2.718
